In [ ]:
# 12/12/2021-1
# https://www.tensorflow.org/hub/tutorials/tf2_image_retraining
from __future__ import absolute_import
from IPython.display import clear_output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from os.path import exists, join

contentPath = os.getcwd()
cv2Path=join(contentPath, 'CV2Images')
testPath=join(contentPath, 'images')
checkpointPath = join(contentPath, 'ChkPoints')
if not exists(checkpointPath):
    os.makedirs(checkpointPath)

import itertools
import glob, os, time
from time import sleep

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

for fil in glob.glob('*.h5'):
    os.remove(fil)

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print('cwd: ', os.getcwd())

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
test_ds = image_dataset_from_directory(
    testPath,
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=1)
print()
# clear_output()

In [ ]:
model_name = "mobilenet_v2_100_224" # @param ['mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

model_handle_map = {
  "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
  "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
  "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
  "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
  "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
  "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
  "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
}

model_image_size_map = {}

model_handle = model_handle_map.get(model_name)
pixels = model_image_size_map.get(model_name, 224)

print(f"Selected model: {model_name} : {model_handle}")

IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 4

In [ ]:
def build_dataset(subset):
    return tf.keras.preprocessing.image_dataset_from_directory(
        cv2Path,
        validation_split=.20,
        subset=subset,
        label_mode="categorical",
        # Seed needs to provided when using validation_split and shuffle = True.
        seed=456,
        image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE)

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

do_data_augmentation = True
if do_data_augmentation:
    preprocessing_model.add(tf.keras.layers.RandomRotation(40))
    preprocessing_model.add(
        tf.keras.layers.RandomTranslation(0, 0.2))
    preprocessing_model.add(
        tf.keras.layers.RandomTranslation(0.2, 0))
    # Like the old tf.keras.preprocessing.image.ImageDataGenerator(),
    # image sizes are fixed when reading, and then a random zoom is applied.
    # If all training inputs are larger than image_size, one could also use
    # RandomCrop with a batch size of 1 and rebatch later.
    preprocessing_model.add(tf.keras.layers.RandomZoom(-0.1, -0.1))
    preprocessing_model.add(
        tf.keras.layers.RandomFlip(mode="horizontal"))
    
train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))
print()
val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

In [ ]:
do_fine_tuning = True

print("Building model with", model_handle)
model = tf.keras.Sequential([
    # Explicitly define the input shape so the model can be properly
    # loaded by the TFLiteConverter
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(model_handle, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(len(class_names),
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005,
                                    momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True,
                                               label_smoothing=0.1),
  metrics=['accuracy'])
print(model.built)
sleep(2)

In [ ]:
earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0, patience=1, verbose=1,
    mode='auto', baseline=None, restore_best_weights=False
)

checkpoints = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointPath,
    monitor='loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)

In [ ]:
steps_per_epoch = train_size // BATCH_SIZE
validation_steps = valid_size // BATCH_SIZE
hist = model.fit(
    train_ds, validation_data=val_ds,
    epochs=10, steps_per_epoch=steps_per_epoch,
    callbacks=[earlyStop, checkpoints],
    validation_steps=validation_steps).history

In [ ]:
with tf.keras.utils.custom_object_scope(custom_objects):
    new_model = keras.models.clone_model(model)

In [ ]:
modelName = model_name + '_retrained.h5'
modelSavePath = join(contentPath, modelName)
# serialize model to JSON
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)
    # print(json_file)

# serialize weights to HDF5
# model.save("model.h5")
model.save(modelSavePath)

print("Saved model to disk")

In [ ]:
model = tf.keras.Sequential([tf.keras.Input((32,)), tf.keras.layers.Dense(1)]
)
config = model.get_config()
new_model = tf.keras.Sequential.from_config(config)
new_model.predict_on_batch(test_ds)

In [ ]:
modelName = model_name + '_retrained.h5'
modelSavePath = join(contentPath, modelName)
model.save(modelName, overwrite=True)
modelSavePath